# 4.5 Custom Layers

## 4.5.1 Layers without Parameters

To start, we construct a custom layer
that does not have any parameters of its own.The following `CenteredLayer` class simply
subtracts the mean from its input.
To build it, we simply need to inherit
from the base layer class and implement the forward propagation function.

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

Let's verify that our layer works as intended by feeding some data through it.

In [2]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

We can now incorporate our layer as a component in constructing more complex models.

In [3]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

/Users/jiayuhuang/anaconda3/envs/d2l/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


As an extra sanity check, we can send random data
through the network and check that the mean is in fact 0.
Because we are dealing with floating point numbers,
we may still see a very small nonzero number
due to quantization.

In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-2.7940e-09, grad_fn=<MeanBackward0>)

## 4.5.2 Layers with Parameters
We can use built-in functions to create parameters, which
provide some basic housekeeping functionality.
In particular, they govern access, initialization,
sharing, saving, and loading model parameters.
This way, among other benefits, we will not need to write
custom serialization routines for every custom layer.

Now let's implement our own version of the  fully connected layer.
Recall that this layer requires two parameters,
one to represent the weight and the other for the bias.
In this implementation, we bake in the ReLU activation as a default.
This layer requires two input arguments: `in_units` and `units`, which
denote the number of inputs and outputs, respectively.

In [5]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

Next, we instantiate the `MyLinear` class
and access its model parameters.

In [6]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 1.1170, -0.4883,  1.8680],
        [ 0.2612, -1.7494, -0.2981],
        [-1.0007, -0.2627,  0.4486],
        [ 1.4781,  0.4477,  0.7075],
        [ 1.3846, -0.1300, -0.9291]], requires_grad=True)

We can directly carry out forward propagation calculations using custom layers.

In [8]:
linear(torch.rand(2, 5))

tensor([[1.4056, 0.0000, 0.0000],
        [3.7758, 0.0319, 0.0000]])

We can also construct models using custom layers.
Once we have that we can use it just like the built-in fully connected layer.


In [9]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[ 7.7653],
        [14.4193]])